In [6]:
import os
import time
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim

In [ ]:
file_path = "DataFrames/DF_Partnerships for the Goals.csv"
df = pd.read_csv(file_path)
# Converte la stringa di lista in una lista vera e propria
df['authors'] = df['authors'].apply(lambda x: eval(x))
df['institutions'] = df['institutions'].apply(lambda x: eval(x))

df.head()

In [ ]:
rows = []

for _, row in df.iterrows():
    authors = row['authors']
    institutions = row['institutions'] if row['institutions'] else [None] * len(authors)
    for author, institution in zip(authors, institutions):
        rows.append({
            'PMID': row['PMID'],
            'Year': row['year'],
            'Author': author.strip(),
            'Institution': institution.strip() if institution else None
        })
df_authors = pd.DataFrame(rows)

df_authors

In [ ]:
df_authors['City'] = None
df_authors['Region'] = None
df_authors['State'] = None
df_authors['Latitude'] = None
df_authors['Longitude'] = None
df_authors.head()

In [2]:
API_KEY = ''

def get_city_state(place_name):
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"address": place_name, "key": API_KEY}
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            city = None
            region = None
            state = None
            latitude = None
            longitude = None
            for component in data['results'][0]['address_components']:
                if "locality" in component['types']:
                    city = component['long_name']
                if not city and "postal_town" in component['types']:
                    city = component['long_name']
                if "administrative_area_level_1" in component['types']:
                    region = component['long_name']
                if "country" in component['types']:
                    state = component['long_name']
                latitude = data["results"][0]["geometry"]["location"]["lat"]
                longitude = data["results"][0]["geometry"]["location"]["lng"]
            return city, region, state, latitude, longitude
    return None, None, None, None, None

def get_city_from_coordinates(lat, lng):
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"latlng": f"{lat},{lng}", "key": API_KEY}  # Latitudine e Longitudine
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            city = None
            region = None
            state = None
            for result in data['results']:
                for component in result['address_components']:
                    if "locality" in component['types']:
                        city = component['long_name']
                    if not city and "postal_town" in component['types']:
                        city = component['long_name']
                    if "administrative_area_level_1" in component['types']:
                        region = component['long_name']
                    if "country" in component['types']:
                        state = component['long_name']
                return city, region, state
    return None, None, None

In [ ]:
for i in range(len(df_authors)):
    if df_authors.loc[i, 'Institution']:
        place_name = df_authors.loc[i, 'Institution']
        city, region, state, latitude, longitude = get_city_state(place_name)
        df_authors.loc[i, 'City'] = city
        df_authors.loc[i, 'Region'] = region
        df_authors.loc[i, 'State'] = state
        df_authors.loc[i, 'Latitude'] = latitude
        df_authors.loc[i, 'Longitude'] = longitude

In [ ]:
for i in range(len(df_authors)):
    print(i)
    lat = df_authors.loc[i, 'Latitude']
    lng = df_authors.loc[i, 'Longitude']
    if pd.notna(lat) and pd.notna(lng):
        city, region, state = get_city_from_coordinates(lat, lng)
        if not df_authors.loc[i, 'City']:
            df_authors.loc[i, 'City'] = city
        if not df_authors.loc[i, 'Region']:
            df_authors.loc[i, 'Region'] = region
        if not df_authors.loc[i, 'State']:
            df_authors.loc[i, 'State'] = state

In [ ]:
df_authors.to_csv('df_Lee12-24_to_pbi.csv', encoding='utf-8', index=False)

In [3]:
import pycountry_convert as pc

def get_continent_from_country(country_name):
    try:
        country_code = pc.country_name_to_country_alpha2(country_name)
        continent_code = pc.country_alpha2_to_continent_code(country_code)
        continent_name = {
            "AF": "Africa",
            "AS": "Asia",
            "EU": "Europe",
            "NA": "North America",
            "SA": "South America",
            "OC": "Oceania",
            "AN": "Antarctica"
        }
        return continent_name.get(continent_code, None)
    except Exception as e:
        return None

Europe
North America
Oceania


In [5]:
source_folder = "-components-of-scientific-productions/DataFrames/"
destination_folder = "Novelty-components-of-scientific-productions/DataFrames_to_PBI/"


for filename in os.listdir(source_folder):
    if filename.endswith(".csv"):
        file_path = os.path.join(source_folder, filename)
        df = pd.read_csv(file_path)

        
        #####
        df['authors'] = df['authors'].apply(lambda x: eval(x))
        df['institutions'] = df['institutions'].apply(lambda x: eval(x))
        
        
        #####
        rows = []
        for _, row in df.iterrows():
            authors = row['authors']
            institutions = row['institutions'] if row['institutions'] else [None] * len(authors)
            for author, institution in zip(authors, institutions):
                rows.append({
                    'PMID': row['PMID'],
                    'Year': row['year'],
                    'Author': author.strip(),
                    'Institution': institution.strip() if institution else None
                })
        df_authors = pd.DataFrame(rows)
        
        #####
        df_authors['City'] = None
        df_authors['Region'] = None
        df_authors['State'] = None
        df_authors['Latitude'] = None
        df_authors['Longitude'] = None
        print(df_authors.head(1))
        
        
        #####
        for i in range(len(df_authors)):
            if df_authors.loc[i, 'Institution']:
                place_name = df_authors.loc[i, 'Institution']
                city, region, state, latitude, longitude = get_city_state(place_name)
                df_authors.loc[i, 'City'] = city
                df_authors.loc[i, 'Region'] = region
                df_authors.loc[i, 'State'] = state
                df_authors.loc[i, 'Latitude'] = latitude
                df_authors.loc[i, 'Longitude'] = longitude
        print("1")
        
        #####
        for i in range(len(df_authors)):
            lat = df_authors.loc[i, 'Latitude']
            lng = df_authors.loc[i, 'Longitude']
            if pd.notna(lat) and pd.notna(lng):
                city, region, state = get_city_from_coordinates(lat, lng)
                if not df_authors.loc[i, 'City']:
                    df_authors.loc[i, 'City'] = city
                if not df_authors.loc[i, 'Region']:
                    df_authors.loc[i, 'Region'] = region
                if not df_authors.loc[i, 'State']:
                    df_authors.loc[i, 'State'] = state
        print("2")
        
        #####
        dest_path = os.path.join(destination_folder, filename)
        df_authors.to_csv(dest_path, encoding='utf-8', index=False)
        print(df_authors.head(1))
        print(f"File {filename} elaborato e salvato in {destination_folder}")

print("Elaborazione completata!")


       PMID  Year         Author                                  Institution  \
0  83458765  2016  Nadja Kabisch  Helmholtz Centre for Environmental Research   

   City Region State Latitude Longitude  
0  None   None  None     None      None  
1
2
       PMID  Year         Author                                  Institution  \
0  83458765  2016  Nadja Kabisch  Helmholtz Centre for Environmental Research   

      City   Region    State   Latitude  Longitude  
0  Leipzig  Sachsen  Germany  51.351752  12.430911  
File DF_Climate Action.csv elaborato e salvato in C:/Users/diego/Documents/GitHub/Novelty-components-of-scientific-productions/DataFrames_to_PBI/
      PMID  Year             Author      Institution  City Region State  \
0  6182150  2016  Endalcachew Bayeh  Ambo University  None   None  None   

  Latitude Longitude  
0     None      None  
1
2
      PMID  Year             Author      Institution  City  Region     State  \
0  6182150  2016  Endalcachew Bayeh  Ambo University 

1
2
       PMID  Year               Author  \
0  48088088  2016  Anastasia Stratigea   

                               Institution      City Region   State  \
0  National Technical University of Athens  Zografou   None  Greece   

    Latitude  Longitude  
0  37.977755  23.783778  
File DF_Sustainable Cities and Communities.csv elaborato e salvato in C:/Users/diego/Documents/GitHub/Novelty-components-of-scientific-productions/DataFrames_to_PBI/
       PMID  Year          Author                      Institution  City  \
0  32852716  2016  Leonie Bellina  Leuphana University of Lüneburg  None   

  Region State Latitude Longitude  
0   None  None     None      None  
1
2
       PMID  Year          Author                      Institution      City  \
0  32852716  2016  Leonie Bellina  Leuphana University of Lüneburg  Lüneburg   

          Region    State   Latitude  Longitude  
0  Niedersachsen  Germany  53.228954  10.401361  
File DF_Zero Hunger.csv elaborato e salvato in C:/Users/dieg

In [11]:
source_folder = "Novelty-components-of-scientific-productions/DataFrames_to_PBI/"
destination_folder = "Novelty-components-of-scientific-productions/DataFrames_to_PBI/"

for filename in os.listdir(source_folder):
    if filename.endswith(".csv"):
        file_path = os.path.join(source_folder, filename)
        df = pd.read_csv(file_path)
        
        df['Continent'] = None
        
        for i in range(len(df)):
            if df.loc[i, 'Institution']:
                df.loc[i, 'Continent'] = get_continent_from_country(df.loc[i, 'State'])
                
        dest_path = os.path.join(destination_folder, filename)
        df.to_csv(dest_path, encoding='utf-8', index=False)
        print(f"File {filename} elaborato e salvato in {destination_folder}")
        
print("Elaborazione completata!")


File DF_Climate Action.csv elaborato e salvato in C:/Users/diego/Desktop/Nuova cartella/
File DF_Gender Equality.csv elaborato e salvato in C:/Users/diego/Desktop/Nuova cartella/
File DF_Life on Land.csv elaborato e salvato in C:/Users/diego/Desktop/Nuova cartella/
File DF_No Poverty.csv elaborato e salvato in C:/Users/diego/Desktop/Nuova cartella/
File DF_Partnerships for the Goals.csv elaborato e salvato in C:/Users/diego/Desktop/Nuova cartella/
File DF_Quality Education.csv elaborato e salvato in C:/Users/diego/Desktop/Nuova cartella/
File DF_Responsible Consumption and Production.csv elaborato e salvato in C:/Users/diego/Desktop/Nuova cartella/
File DF_SDG 14.csv elaborato e salvato in C:/Users/diego/Desktop/Nuova cartella/
File DF_SDG 16.csv elaborato e salvato in C:/Users/diego/Desktop/Nuova cartella/
File DF_SDG 3.csv elaborato e salvato in C:/Users/diego/Desktop/Nuova cartella/
File DF_SDG 6.csv elaborato e salvato in C:/Users/diego/Desktop/Nuova cartella/
File DF_SDG 7.csv ela